In [1]:
import boto3

In [2]:
# two options... can create boto3 client OR resource with DynamoDB

In [3]:
ddb = boto3.client('dynamodb', endpoint_url='http://localhost:8000')
ddb.list_tables()

{'TableNames': [],
 'ResponseMetadata': {'RequestId': 'c5fe2eca-44f2-4ca5-ab56-a0c951563328',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 05 May 2022 15:13:11 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '1315925753',
   'x-amzn-requestid': 'c5fe2eca-44f2-4ca5-ab56-a0c951563328',
   'content-length': '17',
   'server': 'Jetty(9.4.18.v20190429)'},
  'RetryAttempts': 0}}

In [8]:
ddb = boto3.resource('dynamodb', endpoint_url='http://localhost:8000')
list(ddb.tables.all()) # prints empty list, as we have no tables

[]


In [10]:
# we will use resources for this tutorial.
type(ddb)

boto3.resources.factory.dynamodb.ServiceResource

In [51]:
# create table using JSON format
table = ddb.create_table(
    TableName = 'Employees',
    # HASH and RANGE together make up a "primary key", which is unique in the table
    # you can have the same HASH multiple times, but with different RANGES (and vice versa?).
    KeySchema = [
        {
            'AttributeName': 'Name',
            'KeyType': 'HASH' # PARITION KEY
        },
        {
            'AttributeName': 'Email', # SORT KEY (optional)
            'KeyType': 'RANGE'
        }
    ],
    AttributeDefinitions = [
        {
            'AttributeName': 'Name',
            'AttributeType': 'S' # S == STRING
        },
        {
            'AttributeName': 'Email',
            'AttributeType': 'S' 
        }
    ],
    # doesn't matter for local instance
    ProvisionedThroughput={ # THE RATE AT WHICH YOU WANT YOUR READ/WRITE CAPACITY TO BE SET TO
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    },
    BillingMode='PAY_PER_REQUEST', # optional argument, does not matter for local instance
    # note that adding a single GSI doubles cost of writing,
    # and you can add a max of 20 GSIs. Replications to GSI table are quick
    # but not instantaneous.
    
    # also, write capacity on GSI table should honestly be HIGHER than 
    # write capacity on main table
    GlobalSecondaryIndexes=[
        {
            'IndexName': 'Country', # MUST BE UNIQUE ONLY FOR THIS TABLE
            # key schema must have at least a partition key. RANGE is optional
            'KeySchema': [
                {
                    'AttributeName': 'Name', # must this be the same as in main table?
                    'KeyType': 'HASH'
                }
        ],
        # This represents attributes that are copied (projected) from main table into
        # secondary index table. These are in addition to PK atrributes and Index key attributes,
        # which are automatically projected.
            'Projection': {
                # can be 'KEYS_ONLY', 'ALL' or "INCLUDE", the latter of which requires a list to be passed elsewhere. See boto3 documentation.
                'ProjectionType': 'ALL'
                # 'NonKeyAttributes': ['string']
            },
            'ProvisionedThroughput': {
                'ReadCapacityUnits': 1,
                'WriteCapacityUnits': 2
            }
        }
    ]
)

print(table)

dynamodb.Table(name='Employees')


In [52]:
# returns key schema
table.key_schema

[{'AttributeName': 'Name', 'KeyType': 'HASH'},
 {'AttributeName': 'Email', 'KeyType': 'RANGE'}]

In [53]:
# returns the table attribute definitions
table.attribute_definitions

[{'AttributeName': 'Name', 'AttributeType': 'S'},
 {'AttributeName': 'Email', 'AttributeType': 'S'}]

In [54]:
# returned provisioned_throughput
table.provisioned_throughput

{'LastIncreaseDateTime': datetime.datetime(1969, 12, 31, 19, 0, tzinfo=tzlocal()),
 'LastDecreaseDateTime': datetime.datetime(1969, 12, 31, 19, 0, tzinfo=tzlocal()),
 'NumberOfDecreasesToday': 0,
 'ReadCapacityUnits': 0,
 'WriteCapacityUnits': 0}

In [55]:
# find GSIs
table.global_secondary_indexes

[{'IndexName': 'Country',
  'KeySchema': [{'AttributeName': 'Name', 'KeyType': 'HASH'}],
  'Projection': {'ProjectionType': 'ALL'},
  'IndexStatus': 'ACTIVE',
  'ProvisionedThroughput': {'ReadCapacityUnits': 0, 'WriteCapacityUnits': 0},
  'IndexSizeBytes': 0,
  'ItemCount': 0,
  'IndexArn': 'arn:aws:dynamodb:ddblocal:000000000000:table/Employees/index/Country'}]

In [44]:
# scan table though it's empty)
results = table.scan()
for item in results:
    print(item)
    

Items
Count
ScannedCount
ResponseMetadata


In [21]:
results["Items"]

[]

In [22]:
results["Count"]

0

In [23]:
results["ScannedCount"]

0

In [24]:
results["ResponseMetadata"]

{'RequestId': 'd9265853-d3eb-4fa5-8678-d08f8a2a02bd',
 'HTTPStatusCode': 200,
 'HTTPHeaders': {'date': 'Thu, 05 May 2022 16:54:38 GMT',
  'content-type': 'application/x-amz-json-1.0',
  'x-amz-crc32': '583912891',
  'x-amzn-requestid': 'd9265853-d3eb-4fa5-8678-d08f8a2a02bd',
  'content-length': '39',
  'server': 'Jetty(9.4.18.v20190429)'},
 'RetryAttempts': 0}

In [ ]:
# Remember that DynamoDB is SCHEMALESS -- we cannot specify
# columns other that the HASH (and maybe RANGE, which is optional).

# If we want to add a new item, we can specify any number of
# attributes there...

In [50]:
# to delete table D=
table.delete()

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'Name',
    'AttributeType': 'S'},
   {'AttributeName': 'Email', 'AttributeType': 'S'}],
  'TableName': 'Employees',
  'KeySchema': [{'AttributeName': 'Name', 'KeyType': 'HASH'},
   {'AttributeName': 'Email', 'KeyType': 'RANGE'}],
  'TableStatus': 'ACTIVE',
  'CreationDateTime': datetime.datetime(2022, 5, 5, 19, 53, 18, 281000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'LastIncreaseDateTime': datetime.datetime(1969, 12, 31, 19, 0, tzinfo=tzlocal()),
   'LastDecreaseDateTime': datetime.datetime(1969, 12, 31, 19, 0, tzinfo=tzlocal()),
   'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 0,
   'WriteCapacityUnits': 0},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:ddblocal:000000000000:table/Employees',
  'BillingModeSummary': {'BillingMode': 'PAY_PER_REQUEST',
   'LastUpdateToPayPerRequestDateTime': datetime.datetime(2022, 5, 5, 19, 53, 18, 281000, tzinfo=tzlocal())},
  'GlobalSecondaryInd

In [47]:
table.wait_until_not_exists() # check that table does not exist